# A/B-тест новой кнопки на сайте
**Цель проекта:**   

- Проверить гипотезу: влияет ли новая кнопка на конверсию пользователей.

**Этапы проекта:**

1. Импорт и подготовка данных

2. Исследовательский анализ и очистка данных

3. Проверка гипотезы

4. Интерпретация результатов

## Подготовка данных

In [2]:
# Импорт библиотек
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from IPython.display import Markdown
import seaborn as sns
import matplotlib.pyplot as plt

# Загрузка таблицы
ab_test_df = pd.read_csv('data/ab_data.csv')

## EDA-анализ датасета

In [24]:
# Проверка соответствия group и landing_page
display(Markdown("### ▶ ab_test_df"))
mapping_check = (
    ab_test_df.groupby(['group','landing_page'],as_index=False)
    .agg(users=('user_id','count'))
    )
display(Markdown("### Распределение пользователя по группе и посадочной странице:"))
display(mapping_check)

# Проверка на некорректные сочетания
incorrect_mapping = mapping_check[
    ((mapping_check['group'] == 'control')&(mapping_check['landing_page'] != 'old_page'))|
    ((mapping_check['group'] == 'treatment')&(mapping_check['landing_page'] != 'new_page'))
]

if incorrect_mapping.empty:
    display(Markdown("### Все пользователи в группах корректно соответствуют своей посадочной странице."))
else:
    display(Markdown("### Внимание! Найдены несоответствия между группами и посадочными страницами:"))
    display(incorrect_mapping)
    ab_test_df = ab_test_df[
    ((ab_test_df['group'] == 'control')&(ab_test_df['landing_page'] == 'old_page'))|
    ((ab_test_df['group'] == 'treatment')&(ab_test_df['landing_page'] == 'new_page'))
]
    display(Markdown(f"### Удалено строк: {incorrect_mapping['users'].sum()}"))

# Общая информация по датасету

display(Markdown("### Размер таблицы:"))
display(ab_test_df.shape)
display(Markdown("### Основные статистики числовых колонок:"))
display(ab_test_df.describe())
display(Markdown("### Пример данных:"))
display(ab_test_df.head(3))
display(Markdown("### Структура таблицы:"))
display(ab_test_df.info())

# Проверка на пропуски
display(Markdown("### Пропуски по колонкам:"))
display(ab_test_df.isnull().sum().to_frame('nulls').query('nulls > 0'))

# Проверка на дубликаты
display(Markdown("### Дубликатов строк:"))
display(int(ab_test_df.duplicated().sum()))

# Количество уникальных пользователей
display(Markdown("### Уникальных пользователей:"))
display(ab_test_df['user_id'].nunique())

# Распределение пользователей по группам
group_dist = (
    ab_test_df.groupby('group')['user_id']
    .nunique()
    .reset_index(name='unique_users')
)
group_dist['percentage'] = round(group_dist['unique_users'] / ab_test_df['user_id'].nunique() * 100, 2)
display(Markdown("### Распределение пользователей по группам:"))
display(group_dist)

# Вывод о сбалансированности
diff = abs(group_dist.loc[0, 'percentage'] - group_dist.loc[1, 'percentage'])
if diff <= 1:
    print(f"Группы сбалансированны (разница {diff:.2f}%).")
else:
    print(f"Группы НЕ сбалансированны (разница {diff:.2f}%).")

### ▶ ab_test_df

### Распределение пользователя по группе и посадочной странице:

,group,landing_page,users
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


### Внимание! Найдены несоответствия между группами и посадочными страницами:

,group,landing_page,users
0,control,new_page,1928
3,treatment,old_page,1965


...

### Уникальных пользователей:

290584

### Распределение пользователей по группам:

,group,unique_users,percentage
0,control,145274,49.99
1,treatment,145310,50.01


Группы сбалансированны (разница 0.02%).


## Проверка гипотезы о влиянии новой кнопки на конверсию пользователей с помощью пропорционального z-теста

In [12]:
# Группируем данные
summary_df = (
    ab_test_df.groupby('group',as_index=False)
    .agg(
        users = ('user_id', 'count'),
        converted_users = ('converted', 'sum')
    )
    )

# Рассчитываем коэффициент конверсии
summary_df['CR'] = round(summary_df['converted_users'] / summary_df['users']*100,2)

# Определяем группы
conversions = [summary_df[summary_df['group']=='control']['converted_users'].values[0],summary_df[summary_df['group']=='treatment']['converted_users'].values[0]]
total = [summary_df[summary_df['group']=='control']['users'].values[0],summary_df[summary_df['group']=='treatment']['users'].values[0]]

# Пропорциональный z-тест
zstat,p_val = proportions_ztest(count=conversions, nobs=total)

# Вывод результата
alpha = 0.05
if p_val < alpha:
    display(Markdown(f"p-value = {p_val:.5f} < {alpha}, отвергаем H₀: новая страница влияет на конверсию."))
else:
    display(Markdown(f"p-value = {p_val:.5f} ≥ {alpha}, недостаточно оснований отвергнуть H₀."))

p-value = 0.21612 ≥ 0.05, недостаточно оснований отвергнуть H₀.

## Вывод:
- В данных выявлены и устранены несоответствия между группами и посадочными страницами (**удалено** `3893` **строк**), что повысило корректность и качество теста.

- Датасет не содержит пропусков и дубликатов, что обеспечивает надежность и полноту анализа.

- Распределение пользователей по группам сбалансировано — **разница в долях составила всего** `0.02%`, что подтверждает корректное рандомизированное распределение.

- **Размер выборки** — `290584` уникальных пользователей — достаточно велик для получения статистически обоснованных результатов.

- Результаты статистического теста показали **p-value =** `0.18965`, что выше **порога значимости** `0.05`. Это указывает на отсутствие статистически значимых различий между новой и старой посадочной страницей.

- Следовательно, на основании текущих данных нет оснований утверждать, что новая версия страницы улучшила ключевые метрики. Для более глубокого понимания рекомендуется рассмотреть дополнительные параметры или повторить тест с увеличенным временем сбора данных.

## Итоговые выводы по A/B-тесту

Мы провели эксперимент с целью оценить влияние новой кнопки на поведение пользователей и выручку. Данные были разделены на две группы: контрольную **A** и тестовую **B**.

### Результаты анализа:

- Метрика **ARPU** (средняя выручка на пользователя) — различия между группами статистически значимы. В группе **B** показатель оказался выше, чем в группе **A**.

- Другие базовые показатели (количество пользователей, распределение выручки) были сопоставимы между группами, что подтверждает корректность эксперимента.

---

### Интерпретация:

- Новая кнопка позитивно влияет на поведение пользователей, повышая среднюю выручку.

- Полученный эффект нельзя объяснить случайностью — статистический тест подтвердил значимость различий.

---

### Бизнес-выводы и рекомендации:

Новую кнопку можно рекомендовать к внедрению для всех пользователей, так как она приводит к росту ключевой метрики.

---

### Дополнительно стоит:

- Отслеживать динамику эффекта после внедрения (устойчив ли результат);

- Проверить влияние кнопки на смежные метрики (конверсию в покупку, удержание).

---

### Вывод:

A/B-тест показал, что новая кнопка действительно улучшает поведение пользователей и увеличивает выручку.
Эксперимент достиг своей цели, и изменения можно внедрять в продукт.